In [ ]:
# Generate Splits for Training, Validation and Testing

In [ ]:
from pathlib import Path
import h5py
from collections import Counter
import numpy as np

In [ ]:
h5_files = list(Path('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/h5_files/').glob('**/*.h5'))

In [ ]:
import h5py
from collections import Counter
import numpy as np
from pathlib import Path

for file in h5_files:
    dataset_name = file.stem
    
    with h5py.File(file, 'r') as f:
        
        # Header
        print("=" * 60)
        print(f"  DATASET: {dataset_name}")
        print("=" * 60)
        
        # File structure
        print("\n  [FILE STRUCTURE]")
        print(f"    Root keys: {list(f.keys())}")
        print(f"    Number of samples: {len(f['sample_ids'])}")
        print(f"    Sample IDs: {f['sample_ids'][()].astype(str)[:5]}... (showing first 5)")
        
        # Annotations
        print("\n  [ANNOTATIONS]")
        annotations = f['annotation'][()].astype(str)
        print(f"    Shape: {annotations.shape}")
        print(f"    Unique classes: {np.unique(annotations)}")
        
        counts = Counter(annotations)
        total = len(annotations)
        max_count = max(counts.values())
        
        print(f"    Class distribution (sorted by count):")
        # Sort by count descending
        for cls, count in counts.most_common():
            pct = 100 * count / total
            # Scale bar to max 20 chars based on max count
            bar_len = int(20 * count / max_count) + 1
            bar = "█" * bar_len
            print(f"      {cls:>25}: {count:>10} ({pct:>7.3f}%) {bar}")
        
        # Image info
        print("\n  [IMAGE DATA]")
        image_key = list(f['data'].keys())[0]
        W, H, C = f['data'][image_key]['image'].shape
        print(f"    Example sample: {image_key}")
        print(f"    Image dimensions: {W} x {H} x {C}")
        print(f"    Total pixels: {W * H:,}")
        
        # Markers
        print("\n  [MARKERS]")
        markers = f['marker_names'][()].astype(str)
        print(f"    Total markers: {len(markers)}")
        if len(markers) <= 10:
            print(f"    Marker list: {list(markers)}")
        else:
            print(f"    First 10: {list(markers[:10])}")
            print(f"    ... and {len(markers) - 10} more")
        
        # Footer
        print("\n" + "-" * 60 + "\n")

In [ ]:
for file in h5_files:
    dataset_name = file.stem
    
    with h5py.File(file, 'r') as f:
        print(dataset_name)
        
        annotations = f['annotation'][()].astype(str)
        print(len(annotations))
        
        counts = Counter(annotations)
        total_dataset = len(annotations)
        
        print(counts)
        
        splits = dict(
            train=0.7,
            val=0.1,
            test=0.2
        )
        
        assert sum([v for k, v in splits.items()]) == 1
        
        train_idxs, val_idxs, test_idxs = [], [], []
        for celltype, total in counts.items():
            
            all_indices = np.where(annotations == celltype)[0]
            
            np.random.seed(42)
            np.random.shuffle(all_indices)
            
            train_size = int(total * splits['train'])
            val_size = int(total * splits['val'])

            train_idx = sorted(all_indices[:train_size])
            val_idx = sorted(all_indices[train_size:train_size + val_size])
            test_idx = sorted(all_indices[train_size + val_size:])
            
            train_idxs.extend(train_idx)
            val_idxs.extend(val_idx)
            test_idxs.extend(test_idx)
            
        assert (len(train_idxs) + len(val_idxs) + len(test_idxs)) == total_dataset
        
        np.savetxt(file.parent / 'train.txt', sorted(train_idxs), fmt='%d')
        np.savetxt(file.parent / 'val.txt', sorted(val_idxs), fmt='%d')
        np.savetxt(file.parent / 'test.txt', sorted(test_idxs), fmt='%d')

In [ ]:
import matplotlib.pyplot as plt

out_dir = Path('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/h5_files')

exclude_dict = dict(
    IMC_Neuroblastoma = ['IF1', 'IF2', 'IF3', 'MPO', 'H3K9Ac', 'H4K12Ac', 'CXCR2', 'IDO', 'clPARP', 'PNMT', 'DNA1', 'Fibronectin', 'FOXP3'],
    IMC_NeuroblastomaMetaCluster = ['IF1', 'IF2', 'IF3', 'MPO', 'H3K9Ac', 'H4K12Ac', 'CXCR2', 'IDO', 'clPARP', 'PNMT', 'DNA1', 'Fibronectin', 'FOXP3'],
    MIBI_TNBC = ['Au', 'B7H3', 'Background', 'C', 'CD163', 'CSF-1R', 'Ca', 'Fe', 'Na', 'P', 'Segmentation', 'SegmentationInterior', 'Si', 'Ta'],
    CODEX_cHL = ['T-bet', 'TCR-g-d', 'FoxP3', 'MMP-9', 'a-SMA', 'EGFR', 'CD56', 'CD2'],
    CODEX_DLBCL = ['CD57', 'CD152', 'CD69', 'Lambda light chain', 'Mast cell tryptase', 'GATA3', 'Tbet', 'FoxP3', 'BCL6', 'CD15'],
    CODEX_DLBCL2 = ['CD57', 'CD152', 'CD69', 'Lambda light chain', 'Mast cell tryptase', 'GATA3', 'Tbet', 'FoxP3', 'BCL6', 'CD15']
)

for file in h5_files:
    dataset_name = file.stem
    print(dataset_name)
    
    with h5py.File(file, 'r') as f:
        marker_names = set(sorted(f['marker_names'][()].astype(str)))

    skipped_marker = set(exclude_dict[dataset_name])
    
    kept_marker = np.array(list(marker_names - skipped_marker))
    print(len(kept_marker))
    
    marker_file = out_dir / f'{dataset_name}/used_markers.txt'
    
    np.savetxt(marker_file, sorted(kept_marker), fmt="%s")

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from einops import rearrange

# sz = 1500

# with h5py.File('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/h5_files/CODEX_cHL/CODEX_cHL.h5', 'r') as f:
#     marker_names = list(f['marker_names'][()].astype(str))
#     W, H, C = f['data']['1']['image'].shape
#     dapi_idx = marker_names.index('DAPI-01')
#         

#     dapi_image = f['data']['1']['image'][W//2-sz:W//2+sz, H//2-sz:H//2+sz, dapi_idx]
#     dapi_image = np.clip(dapi_image / np.percentile(dapi_image, 99.9), 0, 1)

#     for marker in tqdm(marker_names):
        
#         curr_image = np.zeros((*dapi_image.shape, 3))
        
#         marker_idx = marker_names.index(marker)
        
#         W, H, C = f['data']['1']['image'].shape
#         image = f['data']['1']['image'][W//2-sz:W//2+sz, H//2-sz:H//2+sz, marker_idx]
#         image = np.clip(image / np.percentile(image, 99.9), 0, 1)
        
#         curr_image[..., 0] = image
#         curr_image[..., 2] = dapi_image 
        
#         plt.figure(figsize=(15, 15))
#         plt.imshow(curr_image, vmin=0, vmax=1)
#         plt.title(marker)
#         plt.axis('off')
#         plt.tight_layout()
#         plt.savefig(f'/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/h5_files/CODEX_cHL/examples_with_dapi/{marker}.png')
#         # plt.show()
#         plt.close()
        
        

# Create output directory if it doesn't exist
import os
output_dir_rgb = '/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/h5_files/CODEX_DLBCL2/examples_with_dapi'
os.makedirs(output_dir_rgb, exist_ok=True)

output_dir = '/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/h5_files/CODEX_DLBCL2/examples'
os.makedirs(output_dir, exist_ok=True)

sz=800

with h5py.File('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/h5_files/CODEX_DLBCL2/CODEX_DLBCL2.h5', 'r') as f:
    
    marker_names = list(f['marker_names'][()].astype(str))
    sample_ids = sorted(f['sample_ids'][()].astype(str))
    
    dapi_idx = marker_names.index('DNA')
    dapi_images = []
    for sample in sample_ids:
        W, H, C = f['data'][sample]['image'].shape
        dapi_image = f['data'][sample]['image'][W//2-sz:W//2+sz, H//2-sz:H//2+sz, dapi_idx]
        dapi_image = np.clip(dapi_image / np.percentile(dapi_image, 99.9), 0, 1)
        dapi_images.append(dapi_image)
        
    dapi_images = np.array(dapi_images)
    dapi_images = rearrange(dapi_images, '(X Y) W H -> (X W) (Y H)', X=2, Y=2)
        
    for marker in tqdm(marker_names):
        
        marker_idx = marker_names.index(marker)

        images, samples = [], []
        for sample in sample_ids:
            W, H, C = f['data'][sample]['image'].shape
            image = f['data'][sample]['image'][W//2-sz:W//2+sz, H//2-sz:H//2+sz, marker_idx]
            image = np.clip(image / np.percentile(image, 99.9), 0, 1)
            images.append(image)
            samples.append(sample)
            
        images = np.array(images)
        images = rearrange(images, '(X Y) W H -> (X W) (Y H)', X=2, Y=2)

        rgb_image = np.zeros((*dapi_images.shape, 3))
        rgb_image[..., 0] = images
        rgb_image[..., 2] = dapi_images

        # Create figure with extra space for sample labels
        fig, ax = plt.subplots(figsize=(20, 20))
        
        # Display image
        im = ax.imshow(rgb_image)
        
        # Create title with marker name
        ax.set_title(f'{marker}', fontsize=14, fontweight='bold', pad=20)
        
        # Add sample names as text annotations on the image
        img_h, img_w = images.shape
        positions = [
            (img_w * 0.02, img_h * 0.02),   # top-left
            (img_w * 0.52, img_h * 0.02),   # top-right
            (img_w * 0.02, img_h * 0.52),   # bottom-left
            (img_w * 0.52, img_h * 0.52),   # bottom-right
        ]
        
        # Add sample names with a simple background for readability
        for (x, y), sample in zip(positions, samples):
            # Add white background box for text
            ax.text(x, y, sample, 
                   fontsize=10, 
                   ha='left', 
                   va='center',
                   color='yellow',
                   fontweight='bold')
        ax.axis('off')
        
        # Adjust layout
        plt.tight_layout()
        
        # Save figure
        plt.savefig(f'{output_dir_rgb}/{marker}.png', bbox_inches='tight')
        #plt.show()
        plt.close()
        
        fig, ax = plt.subplots(figsize=(20, 20))
        
        # Display image
        im = ax.imshow(images, cmap='gray')
        
        # Create title with marker name
        ax.set_title(f'{marker}', fontsize=14, fontweight='bold', pad=20)
        
        # Add sample names as text annotations on the image
        img_h, img_w = images.shape
        positions = [
            (img_w * 0.02, img_h * 0.02),   # top-left
            (img_w * 0.52, img_h * 0.02),   # top-right
            (img_w * 0.02, img_h * 0.52),   # bottom-left
            (img_w * 0.52, img_h * 0.52),   # bottom-right
        ]
        
        # Add sample names with a simple background for readability
        for (x, y), sample in zip(positions, samples):
            # Add white background box for text
            ax.text(x, y, sample, 
                   fontsize=10, 
                   ha='left', 
                   va='center',
                   color='yellow',
                   fontweight='bold')
        ax.axis('off')
        
        # Adjust layout
        plt.tight_layout()
        
        # Save figure
        plt.savefig(f'{output_dir}/{marker}.png', bbox_inches='tight')
        #plt.show()
        plt.close()

In [ ]:
# Collapse Fine-Grained IMC Cell Annotations into Broad Meta-Clusters and Overwrite Them in an HDF5 File

# import h5py
# import numpy as np

# meta_cluster_map = {
#     # Tumor Cells
#     'early NB TC': 'Tumor',    'CXCR4hi TC': 'Tumor',
#     'Ki67hi TC': 'Tumor',    'bridge-like TC': 'Tumor',
#     'CD24- marker-lo TC': 'Tumor',    'CD24+ marker-lo TC': 'Tumor',
#     'GD2lo TC': 'Tumor',    'GATA3hi TC': 'Tumor',
#     'CHGAhi TC': 'Tumor',    'LIN- Ki67+': 'Tumor',
    
#     # Stromal
#     'fibroblast/endothel': 'Stromal',    'schwann cell': 'Stromal',
    
#     # T Cells
#     'CD44+ TC': 'T_Cell',
#     'dense T cell region': 'T_Cell',    'CD3+ T cell': 'T_Cell',
#     'CD8+ naive T cell': 'T_Cell',    'CD8+ PDL1lo T cell': 'T_Cell',
#     'CD4+ naive T cell': 'T_Cell',    'CD4+ PD1+ T cell': 'T_Cell',
#     'CD8+ GZMB+ PD1lo T cell': 'T_Cell',    'CD3+ GZMB+ T cell': 'T_Cell',
#     'CD8+ S100B+ T cell': 'T_Cell',
    
#     # B Cells
#     'B cell': 'B_Cell',
    
#     # Myeloid
#     'neutrophil': 'Myeloid',    'monoblast-like': 'Myeloid',
#     'CD14- PDL1+ MO/DC': 'Myeloid',    'CD14+ PDL1+ MO': 'Myeloid',
#     'CD14+ PDL1- MO': 'Myeloid',    'GZMB+ S100B- DC/NK': 'Myeloid',
#     'pDC': 'Myeloid',
    
#     # Progenitors
#     'HPC': 'Progenitor',
#     'neural progenitor': 'Progenitor',    'Ki67+ CXCR4+ cell': 'Progenitor',
    
#     # Other
#     'other': 'Other'
# }

# file_path = '/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/IMC_Neuroblastoma/IMC_NeuroblastomaMetaCluster.h5'

# with h5py.File(file_path, 'r+') as f:
    
#     # Read original annotation
#     annotation = f['annotation'][()].astype(str)
#     print(f"Read {len(annotation)} annotations")
    
#     # Delete old dataset
#     del f['annotation']
#     print("Deleted old 'annotation' dataset")
    
#     # Create meta-annotations
#     meta_annotations = np.array([meta_cluster_map[ann] for ann in annotation])
    
#     # Create new dataset with meta-clusters
#     dt = h5py.special_dtype(vlen=str)
#     meta_annotations_ds = f.create_dataset('annotation', (len(meta_annotations),), dtype=dt)
#     meta_annotations_ds[:] = meta_annotations
    
#     print(f"Created new 'annotation' with meta-clusters: {np.unique(meta_annotations)}")

In [ ]:
# Extract and Match Short Sample IDs from an HDF5 File to Filter PT Tissue Annotations

# import re
# import pandas as pd
# import h5py

# annotations = pd.read_csv('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/Publication/pseudoanonymization-codes.csv', index_col=0)
# file_path = '/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/IMC_Neuroblastoma/IMC_NeuroblastomaMetaCluster.h5'

# sample_ids_short = []

# with h5py.File(file_path, 'r') as f:
#     sample_ids = f['sample_ids'][()].astype(str)
    
#     for f in sample_ids:

#         m = re.search(r'\b\d{2}-\d{3}\b', f)

#         result = m.group(0) if m else None
#         sample_ids_short.append(result)
        
# annotations[annotations.Sample_ID.isin(sample_ids_short) & (annotations.Tissue == 'PT')].shape

In [ ]:
# visualize markers in datasets

from pathlib import Path
import matplotlib.pyplot as plt

# marker_images = list(Path('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/h5_files/CODEX_cHL/examples').glob('*.png'))

# for marker_image in marker_images:
#     plt.figure(figsize=(10,10))
#     plt.imshow(plt.imread(marker_image)[150:-150, 150:-150])
#     plt.axis('off')
#     plt.show()
    
marker_images = list(Path('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/h5_files/CODEX_cHL/examples_with_dapi').glob('*.png'))

for marker_image in marker_images:
    plt.figure(figsize=(10,10))
    plt.imshow(plt.imread(marker_image))
    plt.axis('off')
    plt.show()
    

In [ ]:
# visualize markers in datasets

from pathlib import Path
import matplotlib.pyplot as plt

# marker_images = list(Path('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/h5_files/CODEX_DLBCL2/examples').glob('*.png'))

# for marker_image in marker_images:
#     plt.figure(figsize=(20, 20))
#     plt.imshow(plt.imread(marker_image))
#     plt.axis('off')
#     plt.show()
    
marker_images = list(Path('/home/simon_g/isilon_images_mnt/10_MetaSystems/MetaSystemsData/_simon/data/MCI_data/h5_files/CODEX_DLBCL2/examples_with_dapi').glob('*.png'))

for marker_image in marker_images:
    plt.figure(figsize=(20, 20))
    plt.imshow(plt.imread(marker_image))
    plt.axis('off')
    plt.show()
    